In [1]:
# store start time to get execution time of entire script
import time
start_time = time.time()

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import csv

df = pd.read_csv('data_cleaned.csv')
df.head()

Unnamed: 0    ID State       City                   agyaddr  xobsyr_0  \
0           1   857    OH  Cleveland  1276 West Third St. #400      2005   
1           2   929    OH  Cleveland  1276 West Third St. #400      2006   
2           3   951    OH  Cleveland  1276 West Third St. #400      2006   
3           4  1032    OH  Cleveland  1276 West Third St. #400      2006   
4           5  1039    OH  Cleveland  1276 West Third St. #400      2004   

   Illicit_Days5  Illicit_Cens5  female_cd  nonwhite_cd  unemplmt_cd  \
0            365              0          0            0            0   
1            354              0          0            0            0   
2            365              0          0            0            0   
3            365              0          0            0            0   
4            365              0          0            0            0   

   prsatx_cd  gvsg_cd  CWSg_0_cd  srprobg_cd  dssg_0_cd  epsg_0_cd  \
0          0        2          0           0          1          1   
1          0        1          0           1          0          1   
2          0        0          0           1          0          0   
3          0        2          0           1          1          1   
4          0        0          0           0          1          0   

   adhdg_0_cd  cdsg_0_cd  cjsig_0_cd  lrig_0_cd  srig_0_cd  SESg_0_cd  \
0           1          1           0          1          2          0   
1           0          1           1          0          1          0   
2           0          0           1          0          1          0   
3           1          1           0          2          1          0   
4           0          1           1          2          1          0   

   r4ag_0_cd  nonillicit_flag  primsev_cd_1  primsev_cd_2  primsev_cd_3  \
0          1                1             1             0             0   
1          2                1             1             0             0   
2          2                1             0             0             1   
3          2                1             0             0             1   
4          2                0             0             1             0   

   primsev_cd_4  primsev_cd_5  primsev_cd_6  B2a_0g  SUDSy_0  SDSl_0  
0             0             0             0       0        0       0  
1             0             0             0       0        3       2  
2             0             0             0       0        2       1  
3             0             0             0       0        2       1  
4             0             0             0       0        0       0

In [3]:
df['Address'] = df['agyaddr'] + ', ' + df['City'] + ', ' + df['State']
addresses = list(df['Address'].unique())
len(addresses)

186

In [4]:
%%time

from opencage.geocoder import OpenCageGeocode
from tqdm.notebook import tqdm

# get longitude and latitude for each unique address
key = '2d4ad4cfcdc44021abfe1e347daa6dc7'
geocoder = OpenCageGeocode(key)

geocoded_addrs = []

for addr in tqdm(addresses):
    results = geocoder.geocode(addr)
    lat = results[0]['geometry']['lat']
    lng = results[0]['geometry']['lng']
    geocoded_addrs.append([addr, lat, lng])
    # print(addr + ' -> (' + str(lat) + ', ' + str(lon) + ')')


CPU times: user 5.16 s, sys: 758 ms, total: 5.91 s
Wall time: 3min 39s


In [5]:
geocoded_addrs_df = pd.DataFrame(geocoded_addrs, columns=['Address', 'lat', 'lng'])
geocoded_addrs_df.head()

Address        lat         lng
0    1276 West Third St. #400, Cleveland, OH  41.501028  -81.697772
1   100 Oakdale Campus - M319, Iowa City, IA  41.708538  -91.592663
2           3130 E Broadway Blvd, Tucson, AZ  32.220280 -110.963330
3          1220 Bardstown Rd, Louisville, KY  38.236398  -85.717815
4  1111 Barranca Dr., Suite 800, El Paso, TX  31.747334 -106.335056

In [6]:
geocoded_addrs_df.isna().sum()

Address    0
lat        0
lng        0
dtype: int64

In [7]:
%%time

import urllib, json, requests
import numpy as np

# get block_fips code based on longitude and latitude
def api_call(row):
    lat = row['lat']
    # http://data.fcc.gov/api/block/find?format=json&latitude=28.35975&longitude=-81.421988&showall=true
    lng = row['lng']
    URL = "https://geo.fcc.gov/api/census/block/find?format=json&latitude=" + str(lat) + "&longitude=" + str(lng)
    with urllib.request.urlopen(URL) as url:
        data = json.loads(url.read().decode())
    try:
        return int(data['County']['FIPS'])
    except TypeError: # response failed to get county fips
        return np.nan

for ind,row in tqdm(geocoded_addrs_df.iterrows(), total=geocoded_addrs_df.shape[0]):
    geocoded_addrs_df.set_value(ind, 'Geo_FIPS', api_call(row))
    
geocoded_addrs_df.head()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



CPU times: user 4.11 s, sys: 588 ms, total: 4.7 s
Wall time: 1min 14s


Address        lat         lng  Geo_FIPS
0    1276 West Third St. #400, Cleveland, OH  41.501028  -81.697772   39035.0
1   100 Oakdale Campus - M319, Iowa City, IA  41.708538  -91.592663   19103.0
2           3130 E Broadway Blvd, Tucson, AZ  32.220280 -110.963330    4019.0
3          1220 Bardstown Rd, Louisville, KY  38.236398  -85.717815   21111.0
4  1111 Barranca Dr., Suite 800, El Paso, TX  31.747334 -106.335056   48141.0

In [8]:
geocoded_addrs_df.isna().sum()

Address     0
lat         0
lng         0
Geo_FIPS    1
dtype: int64

In [9]:
# re-merge patient data and geocoded addresses by the address feature
df = pd.merge(df, geocoded_addrs_df, on=['Address'], how='left')

In [10]:
df.shape

(21264, 38)

In [11]:
df.head()

Unnamed: 0    ID State       City                   agyaddr  xobsyr_0  \
0           1   857    OH  Cleveland  1276 West Third St. #400      2005   
1           2   929    OH  Cleveland  1276 West Third St. #400      2006   
2           3   951    OH  Cleveland  1276 West Third St. #400      2006   
3           4  1032    OH  Cleveland  1276 West Third St. #400      2006   
4           5  1039    OH  Cleveland  1276 West Third St. #400      2004   

   Illicit_Days5  Illicit_Cens5  female_cd  nonwhite_cd  unemplmt_cd  \
0            365              0          0            0            0   
1            354              0          0            0            0   
2            365              0          0            0            0   
3            365              0          0            0            0   
4            365              0          0            0            0   

   prsatx_cd  gvsg_cd  CWSg_0_cd  srprobg_cd  dssg_0_cd  epsg_0_cd  \
0          0        2          0           0          1          1   
1          0        1          0           1          0          1   
2          0        0          0           1          0          0   
3          0        2          0           1          1          1   
4          0        0          0           0          1          0   

   adhdg_0_cd  cdsg_0_cd  cjsig_0_cd  lrig_0_cd  srig_0_cd  SESg_0_cd  \
0           1          1           0          1          2          0   
1           0          1           1          0          1          0   
2           0          0           1          0          1          0   
3           1          1           0          2          1          0   
4           0          1           1          2          1          0   

   r4ag_0_cd  nonillicit_flag  primsev_cd_1  primsev_cd_2  primsev_cd_3  \
0          1                1             1             0             0   
1          2                1             1             0             0   
2          2                1             0             0             1   
3          2                1             0             0             1   
4          2                0             0             1             0   

   primsev_cd_4  primsev_cd_5  primsev_cd_6  B2a_0g  SUDSy_0  SDSl_0  \
0             0             0             0       0        0       0   
1             0             0             0       0        3       2   
2             0             0             0       0        2       1   
3             0             0             0       0        2       1   
4             0             0             0       0        0       0   

                                   Address        lat        lng  Geo_FIPS  
0  1276 West Third St. #400, Cleveland, OH  41.501028 -81.697772   39035.0  
1  1276 West Third St. #400, Cleveland, OH  41.501028 -81.697772   39035.0  
2  1276 West Third St. #400, Cleveland, OH  41.501028 -81.697772   39035.0  
3  1276 West Third St. #400, Cleveland, OH  41.501028 -81.697772   39035.0  
4  1276 West Third St. #400, Cleveland, OH  41.501028 -81.697772   39035.0

In [12]:
df.to_csv('data_geocoded.csv')

In [13]:
# print out total notebook execution time
total_seconds = int(time.time() - start_time)
minutes = total_seconds // 60
seconds = total_seconds % 60
print("--- " + str(minutes) + " minutes " + str(seconds) + " seconds ---")

--- 4 minutes 57 seconds ---
